In [ ]:
# nb41_state_machine_core.ipynb
# Goal: Build core state machine for text-based adventure game
# Stage 5: RAG × Text-Adventure

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# =============================================================================
# Cell 2: Core Data Structures
# =============================================================================

from dataclasses import dataclass, field
from typing import Dict, List, Any, Optional, Callable
import json
import random
from enum import Enum


class ActionType(Enum):
    """Action types for choice effects"""

    SET_FLAG = "set_flag"
    MODIFY_STAT = "modify_stat"
    ADD_ITEM = "add_item"
    REMOVE_ITEM = "remove_item"
    GOTO_SCENE = "goto_scene"
    RANDOM_EVENT = "random_event"


@dataclass
class GameState:
    """Complete game state container"""

    current_scene: str = "start"
    chapter: int = 1
    turn: int = 1

    # Player stats
    hp: int = 100
    mp: int = 50
    gold: int = 100

    # Inventory and flags
    inventory: List[str] = field(default_factory=list)
    flags: Dict[str, Any] = field(default_factory=dict)

    # Game metadata
    seed: Optional[int] = None
    total_turns: int = 0

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for serialization"""
        return {
            "current_scene": self.current_scene,
            "chapter": self.chapter,
            "turn": self.turn,
            "hp": self.hp,
            "mp": self.mp,
            "gold": self.gold,
            "inventory": self.inventory.copy(),
            "flags": self.flags.copy(),
            "seed": self.seed,
            "total_turns": self.total_turns,
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "GameState":
        """Create from dictionary"""
        return cls(**data)


@dataclass
class Choice:
    """Represents a player choice option"""

    id: str
    text: str
    description: str = ""

    # Requirements to show this choice
    required_flags: Dict[str, Any] = field(default_factory=dict)
    required_items: List[str] = field(default_factory=list)
    min_stats: Dict[str, int] = field(default_factory=dict)

    # Effects when chosen
    effects: List[Dict[str, Any]] = field(default_factory=list)
    success_text: str = ""
    failure_text: str = ""

    def is_available(self, state: GameState) -> bool:
        """Check if choice is available given current state"""
        # Check flags
        for flag, value in self.required_flags.items():
            if state.flags.get(flag) != value:
                return False

        # Check items
        for item in self.required_items:
            if item not in state.inventory:
                return False

        # Check stats
        for stat, min_val in self.min_stats.items():
            if getattr(state, stat, 0) < min_val:
                return False

        return True


@dataclass
class Scene:
    """Represents a game scene/location"""

    id: str
    title: str
    description: str
    choices: List[Choice] = field(default_factory=list)

    # Scene conditions
    required_flags: Dict[str, Any] = field(default_factory=dict)
    one_time_only: bool = False

    # Narrative elements
    first_visit_text: str = ""
    repeat_visit_text: str = ""

    def get_description(self, state: GameState) -> str:
        """Get appropriate description based on visit history"""
        scene_key = f"visited_{self.id}"
        if state.flags.get(scene_key, False) and self.repeat_visit_text:
            return self.repeat_visit_text
        elif not state.flags.get(scene_key, False) and self.first_visit_text:
            return self.first_visit_text
        else:
            return self.description

In [ ]:
# =============================================================================
# Cell 3: State Machine Engine
# =============================================================================


class StateMachine:
    """Core state machine for text adventure game"""

    def __init__(self, initial_state: Optional[GameState] = None):
        self.state = initial_state or GameState()
        self.scenes: Dict[str, Scene] = {}
        self.random = random.Random(self.state.seed)

        # History tracking
        self.action_history: List[Dict[str, Any]] = []
        self.scene_history: List[str] = []

    def register_scene(self, scene: Scene):
        """Register a new scene"""
        self.scenes[scene.id] = scene
        print(f"[StateMachine] Registered scene: {scene.id}")

    def get_current_scene(self) -> Optional[Scene]:
        """Get current scene object"""
        return self.scenes.get(self.state.current_scene)

    def get_available_choices(self) -> List[Choice]:
        """Get all available choices for current scene"""
        scene = self.get_current_scene()
        if not scene:
            return []

        available = []
        for choice in scene.choices:
            if choice.is_available(self.state):
                available.append(choice)

        return available

    def execute_action(self, action: Dict[str, Any]) -> str:
        """Execute a single action and return result message"""
        action_type = ActionType(action["type"])
        result_msg = ""

        if action_type == ActionType.SET_FLAG:
            flag = action["flag"]
            value = action["value"]
            self.state.flags[flag] = value
            result_msg = f"設定標記 {flag} = {value}"

        elif action_type == ActionType.MODIFY_STAT:
            stat = action["stat"]
            change = action["change"]
            old_val = getattr(self.state, stat, 0)
            new_val = max(0, old_val + change)  # Prevent negative stats
            setattr(self.state, stat, new_val)
            result_msg = f"{stat}: {old_val} → {new_val} ({change:+d})"

        elif action_type == ActionType.ADD_ITEM:
            item = action["item"]
            if item not in self.state.inventory:
                self.state.inventory.append(item)
                result_msg = f"獲得物品：{item}"
            else:
                result_msg = f"已持有：{item}"

        elif action_type == ActionType.REMOVE_ITEM:
            item = action["item"]
            if item in self.state.inventory:
                self.state.inventory.remove(item)
                result_msg = f"失去物品：{item}"
            else:
                result_msg = f"未持有：{item}"

        elif action_type == ActionType.GOTO_SCENE:
            new_scene = action["scene"]
            if new_scene in self.scenes:
                self.scene_history.append(self.state.current_scene)
                self.state.current_scene = new_scene
                result_msg = f"前往：{new_scene}"
            else:
                result_msg = f"錯誤：場景 {new_scene} 不存在"

        return result_msg

    def make_choice(self, choice_id: str) -> Dict[str, Any]:
        """Process player choice and return result"""
        scene = self.get_current_scene()
        if not scene:
            return {"success": False, "message": "當前場景不存在", "effects": []}

        # Find the choice
        choice = None
        for c in scene.choices:
            if c.id == choice_id:
                choice = c
                break

        if not choice:
            return {
                "success": False,
                "message": f"選項 {choice_id} 不存在",
                "effects": [],
            }

        if not choice.is_available(self.state):
            return {"success": False, "message": "此選項目前無法使用", "effects": []}

        # Execute effects
        effect_results = []
        success = True

        for effect in choice.effects:
            try:
                result_msg = self.execute_action(effect)
                effect_results.append(result_msg)
            except Exception as e:
                effect_results.append(f"執行錯誤：{str(e)}")
                success = False

        # Mark scene as visited
        scene_key = f"visited_{scene.id}"
        self.state.flags[scene_key] = True

        # Update game state
        self.state.turn += 1
        self.state.total_turns += 1

        # Record action
        action_record = {
            "turn": self.state.turn - 1,
            "scene": scene.id,
            "choice": choice_id,
            "choice_text": choice.text,
            "effects": effect_results,
            "timestamp": self.state.total_turns,
        }
        self.action_history.append(action_record)

        return {
            "success": success,
            "message": choice.success_text if success else choice.failure_text,
            "effects": effect_results,
            "action_record": action_record,
        }

In [ ]:
# =============================================================================
# Cell 4: Scene Definition System
# =============================================================================


class SceneBuilder:
    """Helper class for building scenes fluently"""

    def __init__(self, scene_id: str):
        self.scene = Scene(id=scene_id, title="", description="")

    def title(self, title: str) -> "SceneBuilder":
        self.scene.title = title
        return self

    def description(self, desc: str) -> "SceneBuilder":
        self.scene.description = desc
        return self

    def first_visit(self, text: str) -> "SceneBuilder":
        self.scene.first_visit_text = text
        return self

    def repeat_visit(self, text: str) -> "SceneBuilder":
        self.scene.repeat_visit_text = text
        return self

    def add_choice(self, choice_id: str, text: str, **kwargs) -> "SceneBuilder":
        choice = Choice(id=choice_id, text=text, **kwargs)
        self.scene.choices.append(choice)
        return self

    def require_flag(self, flag: str, value: Any) -> "SceneBuilder":
        self.scene.required_flags[flag] = value
        return self

    def one_time(self) -> "SceneBuilder":
        self.scene.one_time_only = True
        return self

    def build(self) -> Scene:
        return self.scene


def create_scene(scene_id: str) -> SceneBuilder:
    """Create a new scene builder"""
    return SceneBuilder(scene_id)


# Scene registry for easy management
class SceneRegistry:
    """Registry for managing game scenes"""

    def __init__(self):
        self.scenes: Dict[str, Scene] = {}

    def register(self, scene: Scene):
        """Register a scene"""
        self.scenes[scene.id] = scene
        print(f"[Registry] Registered: {scene.id} - {scene.title}")

    def get(self, scene_id: str) -> Optional[Scene]:
        """Get a scene by ID"""
        return self.scenes.get(scene_id)

    def list_scenes(self) -> List[str]:
        """List all registered scene IDs"""
        return list(self.scenes.keys())

    def export_to_state_machine(self, sm: StateMachine):
        """Export all scenes to a state machine"""
        for scene in self.scenes.values():
            sm.register_scene(scene)

In [ ]:
# =============================================================================
# Cell 5: Choice Processing Logic
# =============================================================================


class ChoiceProcessor:
    """Advanced choice processing with validation and effects"""

    @staticmethod
    def create_stat_effect(stat: str, change: int) -> Dict[str, Any]:
        """Create a stat modification effect"""
        return {"type": ActionType.MODIFY_STAT.value, "stat": stat, "change": change}

    @staticmethod
    def create_item_effect(item: str, add: bool = True) -> Dict[str, Any]:
        """Create an item add/remove effect"""
        return {
            "type": ActionType.ADD_ITEM.value if add else ActionType.REMOVE_ITEM.value,
            "item": item,
        }

    @staticmethod
    def create_flag_effect(flag: str, value: Any) -> Dict[str, Any]:
        """Create a flag setting effect"""
        return {"type": ActionType.SET_FLAG.value, "flag": flag, "value": value}

    @staticmethod
    def create_goto_effect(scene: str) -> Dict[str, Any]:
        """Create a scene transition effect"""
        return {"type": ActionType.GOTO_SCENE.value, "scene": scene}

    @staticmethod
    def validate_choice_data(choice_data: Dict[str, Any]) -> bool:
        """Validate choice data structure"""
        required_fields = ["id", "text"]
        for field in required_fields:
            if field not in choice_data:
                print(f"[Validation] Missing required field: {field}")
                return False
        return True


# Common choice templates
class ChoiceTemplates:
    """Pre-defined choice templates for common actions"""

    @staticmethod
    def combat_choice(target: str, damage: int, cost_mp: int = 0) -> Choice:
        """Create a combat choice"""
        effects = []
        if cost_mp > 0:
            effects.append(ChoiceProcessor.create_stat_effect("mp", -cost_mp))

        return Choice(
            id=f"attack_{target.lower()}",
            text=f"攻擊 {target}",
            description=f"對 {target} 發動攻擊，造成 {damage} 點傷害",
            min_stats={"mp": cost_mp} if cost_mp > 0 else {},
            effects=effects,
            success_text=f"成功攻擊 {target}！",
            failure_text=f"攻擊 {target} 失敗...",
        )

    @staticmethod
    def explore_choice(destination: str, requirements: Dict[str, Any] = None) -> Choice:
        """Create an exploration choice"""
        return Choice(
            id=f"goto_{destination.lower()}",
            text=f"前往 {destination}",
            description=f"探索 {destination} 區域",
            required_flags=requirements or {},
            effects=[ChoiceProcessor.create_goto_effect(destination)],
            success_text=f"你來到了 {destination}",
            failure_text="無法前往該地點",
        )

    @staticmethod
    def item_choice(item: str, cost: int = 0) -> Choice:
        """Create an item acquisition choice"""
        effects = [ChoiceProcessor.create_item_effect(item, True)]
        if cost > 0:
            effects.append(ChoiceProcessor.create_stat_effect("gold", -cost))

        return Choice(
            id=f"get_{item.lower()}",
            text=f"獲取 {item}" + (f" (花費 {cost} 金幣)" if cost > 0 else ""),
            description=f"將 {item} 加入背包",
            min_stats={"gold": cost} if cost > 0 else {},
            effects=effects,
            success_text=f"獲得了 {item}！",
            failure_text=f"無法獲得 {item}",
        )

In [ ]:
# =============================================================================
# Cell 6: Game Loop Implementation
# =============================================================================


class GameLoop:
    """Main game loop controller"""

    def __init__(self, state_machine: StateMachine):
        self.sm = state_machine
        self.running = False
        self.max_turns = 1000  # Safety limit

    def display_scene(self) -> str:
        """Display current scene information"""
        scene = self.sm.get_current_scene()
        if not scene:
            return "錯誤：找不到當前場景"

        output = []
        output.append("=" * 50)
        output.append(f"📍 {scene.title}")
        output.append("=" * 50)
        output.append(scene.get_description(self.sm.state))
        output.append("")

        # Display stats
        state = self.sm.state
        output.append(f"💚 HP: {state.hp} | 💙 MP: {state.mp} | 💰 Gold: {state.gold}")
        output.append(
            f"🎒 背包: {', '.join(state.inventory) if state.inventory else '空'}"
        )
        output.append(f"📖 章節: {state.chapter} | 🎯 回合: {state.turn}")
        output.append("")

        return "\n".join(output)

    def display_choices(self) -> str:
        """Display available choices"""
        choices = self.sm.get_available_choices()
        if not choices:
            return "沒有可用的選項。"

        output = ["可用選項："]
        for i, choice in enumerate(choices, 1):
            prefix = f"[{i}] {choice.text}"
            if choice.description:
                prefix += f" - {choice.description}"
            output.append(prefix)

        return "\n".join(output)

    def process_turn(self, choice_input: str) -> Dict[str, Any]:
        """Process a single game turn"""
        choices = self.sm.get_available_choices()

        # Parse input (number or choice ID)
        selected_choice = None

        if choice_input.isdigit():
            idx = int(choice_input) - 1
            if 0 <= idx < len(choices):
                selected_choice = choices[idx]
        else:
            # Try to find by ID
            for choice in choices:
                if choice.id == choice_input:
                    selected_choice = choice
                    break

        if not selected_choice:
            return {"success": False, "message": "無效的選項", "turn_info": {}}

        # Execute choice
        result = self.sm.make_choice(selected_choice.id)

        return {
            "success": result["success"],
            "message": result["message"],
            "effects": result.get("effects", []),
            "turn_info": {
                "turn": self.sm.state.turn,
                "scene": self.sm.state.current_scene,
                "choice": selected_choice.text,
            },
        }

    def run_interactive(self, max_turns: int = 10):
        """Run interactive game loop (for notebook demo)"""
        self.running = True
        turn_count = 0

        print("🎮 文字冒險遊戲開始！")
        print(self.display_scene())

        while self.running and turn_count < max_turns:
            print(self.display_choices())

            # For demo, we'll simulate choices
            choices = self.sm.get_available_choices()
            if not choices:
                print("遊戲結束：沒有可用選項")
                break

            # Auto-select first choice for demo
            choice_input = "1"
            print(f"\n> 選擇: {choice_input}")

            result = self.process_turn(choice_input)

            if result["success"]:
                print(f"✅ {result['message']}")
                if result.get("effects"):
                    for effect in result["effects"]:
                        print(f"   • {effect}")
            else:
                print(f"❌ {result['message']}")

            print(f"\n--- 回合 {result['turn_info']['turn']} 結束 ---\n")
            print(self.display_scene())

            turn_count += 1

            # Stop condition
            if self.sm.state.current_scene == "ending":
                print("🎉 達成結局！")
                break

        print(f"\n遊戲結束！總共進行了 {turn_count} 回合。")
        return self.sm.state

In [ ]:
# =============================================================================
# Cell 7: Sample Scenes Definition
# =============================================================================


def create_demo_scenes() -> SceneRegistry:
    """Create a set of demo scenes for testing"""
    registry = SceneRegistry()

    # Starting scene
    start_scene = (
        create_scene("start")
        .title("神秘森林入口")
        .description(
            "你站在一片茂密森林的邊緣，古老的樹木遮天蔽日，林間傳來不明的聲響。"
        )
        .first_visit("這是你第一次來到這個神秘的地方，空氣中瀰漫著魔法的氣息。")
        .repeat_visit("你再次來到熟悉的森林入口。")
        .add_choice(
            "explore_forest",
            "深入森林",
            description="冒險進入森林深處",
            effects=[ChoiceProcessor.create_goto_effect("forest_deep")],
        )
        .add_choice(
            "check_equipment",
            "檢查裝備",
            description="整理背包並檢查狀態",
            effects=[ChoiceProcessor.create_stat_effect("hp", 10)],
        )
        .add_choice(
            "find_treasure",
            "尋找寶藏",
            description="在附近搜索可能的寶物",
            effects=[
                ChoiceProcessor.create_item_effect("forest_herb", True),
                ChoiceProcessor.create_flag_effect("found_first_treasure", True),
            ],
        )
        .build()
    )

    # Deep forest scene
    forest_scene = (
        create_scene("forest_deep")
        .title("森林深處")
        .description("你深入森林，周圍的樹木更加高大，陽光幾乎無法穿透樹冠。")
        .add_choice(
            "meet_wizard",
            "遇見巫師",
            description="一位神秘的巫師出現在你面前",
            effects=[
                ChoiceProcessor.create_flag_effect("met_wizard", True),
                ChoiceProcessor.create_goto_effect("wizard_hut"),
            ],
        )
        .add_choice(
            "fight_monster",
            "戰鬥野獸",
            description="與森林野獸戰鬥",
            min_stats={"hp": 30},
            effects=[
                ChoiceProcessor.create_stat_effect("hp", -20),
                ChoiceProcessor.create_stat_effect("gold", 50),
                ChoiceProcessor.create_flag_effect("defeated_monster", True),
            ],
            success_text="你擊敗了野獸！",
            failure_text="你體力不足，無法戰鬥。",
        )
        .add_choice(
            "return_entrance",
            "返回入口",
            description="返回森林入口",
            effects=[ChoiceProcessor.create_goto_effect("start")],
        )
        .build()
    )

    # Wizard hut scene
    wizard_scene = (
        create_scene("wizard_hut")
        .title("巫師小屋")
        .description("一間樸素的小木屋，裡面擺滿了各種奇異的物品和魔法書籍。")
        .add_choice(
            "learn_magic",
            "學習魔法",
            description="向巫師學習基礎魔法",
            required_flags={"met_wizard": True},
            effects=[
                ChoiceProcessor.create_stat_effect("mp", 20),
                ChoiceProcessor.create_item_effect("magic_scroll", True),
                ChoiceProcessor.create_flag_effect("learned_magic", True),
            ],
        )
        .add_choice(
            "buy_potion",
            "購買藥水",
            description="花費金幣購買治療藥水",
            min_stats={"gold": 30},
            effects=[
                ChoiceProcessor.create_stat_effect("gold", -30),
                ChoiceProcessor.create_item_effect("health_potion", True),
            ],
        )
        .add_choice(
            "ask_quest",
            "詢問任務",
            description="向巫師詢問是否有任務",
            effects=[
                ChoiceProcessor.create_flag_effect("got_quest", True),
                ChoiceProcessor.create_goto_effect("quest_area"),
            ],
        )
        .build()
    )

    # Quest area (ending scene for demo)
    quest_scene = (
        create_scene("quest_area")
        .title("任務地點")
        .description("你來到巫師指定的任務地點，這裡似乎隱藏著重要的秘密。")
        .add_choice(
            "complete_quest",
            "完成任務",
            description="執行巫師交付的神秘任務",
            required_flags={"got_quest": True},
            effects=[
                ChoiceProcessor.create_stat_effect("gold", 100),
                ChoiceProcessor.create_flag_effect("quest_completed", True),
                ChoiceProcessor.create_goto_effect("ending"),
            ],
        )
        .add_choice(
            "return_wizard",
            "返回巫師",
            description="回到巫師小屋",
            effects=[ChoiceProcessor.create_goto_effect("wizard_hut")],
        )
        .build()
    )

    # Ending scene
    ending_scene = (
        create_scene("ending")
        .title("冒險結束")
        .description("恭喜！你成功完成了這次冒險。巫師對你的表現非常滿意。")
        .add_choice(
            "restart",
            "重新開始",
            description="重新開始新的冒險",
            effects=[ChoiceProcessor.create_goto_effect("start")],
        )
        .build()
    )

    # Register all scenes
    for scene in [start_scene, forest_scene, wizard_scene, quest_scene, ending_scene]:
        registry.register(scene)

    return registry

In [ ]:
# =============================================================================
# Cell 8: Smoke Test - Complete Turn Verification
# =============================================================================

def smoke_test_state_machine():
    """Comprehensive smoke test for the state machine"""
    print("🧪 State Machine Smoke Test")
    print("=" * 40)

    # Initialize
    initial_state = GameState(seed=42)
    sm = StateMachine(initial_state)

    # Create and load demo scenes
    registry = create_demo_scenes()
    registry.export_to_state_machine(sm)

    print(f"✅ Scenes loaded: {len(sm.scenes)}")
    print(f"✅ Starting scene: {sm.state.current_scene}")
    print(f"✅ Initial HP: {sm.state.hp}")

    # Test 1: Display current scene
    scene = sm.get_current_scene()
    assert scene is not None, "Current scene should exist"
    print(f"✅ Current scene: {scene.title}")

    # Test 2: Get available choices
    choices = sm.get_available_choices()
    assert len(choices) > 0, "Should have available choices"